# This implementation of the genetic algorithm for the science objectives and is how the optimal solution was fount

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [29]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime

# Loads in the analog payloads and the science objective scores

In [30]:
#import cell

#science
science_array = []

#take data from experiments csv and convert it to an array
with open('Found_Experiments.csv', 'rt') as f:
    array_y = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_y:
        science_array.append(line)
#print(science_array)


#take data from scores science csv and convert it to an array
objectives_score = []
with open('Objective_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        objectives_score.append(line)


# Prepares data from the .csv files and sets parameter values

In [ ]:
#data processing cell

#Parse objective data
science_array_array_mod = science_array[1:]                                         #cut off title score
science_mass_array      = np.asarray([i[-1] for i in science_array_array_mod],float)      #Masses of experiments 0 indexed
science_power_array     = np.asarray([i[-2] for i in science_array_array_mod],float)      #power consumption of experiments 0 indexed
objective_array_temp    = [np.array(i[1:-2]) for i in science_array_array_mod]  
objective_array         = []
for i in objective_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    objective_array.append(arr)
objective           = np.array(objectives_score[1:])
objective           = np.asarray(objective[0:, 1:],float)
objective_matrix    = np.zeros((len(objective_array),len(objective)))
for i in range(len(objective_matrix)):
    for j in objective_array[i]:
        objective_matrix[i][j-1] = 1

objective_matrix = np.transpose(objective_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

objective_Understanding = objective[0:, 1]
objective_Humans        = objective[0:, 2]

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function being optimized

In [32]:
#score function for science

def score_science(mission, check_mass=True):
    #weights
    w1 = 4  #completed_score
    w2 = 5  #understand_score
    w3 = 7  #human_score

    mass_max_science = max_mass*mass_ratio #in kg
    mass_min_science = min_mass*mass_ratio #in kg

    #Mass check
    if check_mass and (science_mass_array.dot(mission) + beta*science_power_array.dot(mission) <= mass_min_science or science_mass_array.dot(mission) + beta*science_power_array.dot(mission) > mass_max_science):
        return abs(science_mass_array.dot(mission) + beta*science_power_array.dot(mission))

    #normlizers
    max_number      = 46       #normalize number score 0-1
    max_understand  = 5        #normlize understanding score 0-1
    max_human       = 5        #normlize human score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(objective_matrix @ mission, a_min=None, a_max=1) #A objective is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of objective completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_understanding_score   = objective_Understanding @ normalized_completed       #Generates the understanding score
    completed_human_score           = objective_Humans   @ normalized_completed            #Generates the human score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_understanding_score/max_understand + w3*completed_human_score/max_human
    return -score


# Initial Guess

In [33]:
#analyitcal solution
scores_per_sci = np.zeros(len(science_array)-1)
for i in range(len(scores_per_sci)):
    input = np.zeros(len(science_array)-1)
    input[i] = 1
    scores_per_sci[i] = score_science(input,check_mass=False)
indcies_tech = np.argsort(scores_per_sci)
scores_per_tech_sort = np.sort(scores_per_sci) 

#rint(scores_per_tech_sort)
#print(indcies_tech)
mass = 0
bring_array_science = np.zeros(len(science_array)-1)
for i in range(len(indcies_tech)):
    if mass < 0.5*(max_mass+min_mass)*(mass_ratio):
        bring_array_science[indcies_tech[i]] = 1
        mass = science_mass_array.dot(bring_array_science) + beta*science_power_array.dot(bring_array_science)

print(bring_array_science)
print(score_science(bring_array_science))
#print(mass)
bounds_science          = Bounds(lb=np.zeros(len(objective_array)), ub=np.ones(len(objective_array)))                                                       #we either bring or we dont

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
-7.619983277591974


# Trackers used to allow for multiple loops

In [34]:
#setup
outputs_sci = []
scores_sci = []
seeds_sci = []

# Differential Evolution loop call

In [ ]:
#loop 3min a loop
for i in range(2000):
    seed = np.random.randint(9999999)
    seeds_sci.append(seed)
    res_science = differential_evolution(func=score_science, bounds=bounds_science, x0=bring_array_science, integrality=np.ones(len(science_array)-1),updating='deferred',tol=0.01,init='sobol',popsize=16,recombination=0.05,mutation=(0.7,1.7),maxiter=1000,seed=seed)
    outputs_sci.append(res_science['x'])
    scores_sci.append(res_science['fun'])

# Plots occurance of analog payloads

In [ ]:
#precentage plotter
percentages = np.mean(outputs_sci, axis=0) * 100
sorted_indices = np.argsort(-percentages)
sorted_percentages = percentages[sorted_indices]
indices = [row[0] for row in science_array_array_mod]
indices = [indices[i] for i in sorted_indices]
plt.figure(figsize=(15,5))
plt.bar(indices, sorted_percentages, color='grey')
plt.ylabel('Occurrence Percentage')
plt.title('Common Experiments')
plt.xticks(indices, rotation=90, fontsize=7)
plt.show()

# Plots seeds with scores 

In [ ]:
#score plotter
plt.figure(figsize=(20,5))
seed_index = [str(i) for i in seeds_sci]
plt.bar(seed_index, np.asfarray(scores_sci), color='grey')
plt.xticks(seed_index, rotation=90, fontsize=7)
plt.ylabel('Scores')
plt.title('Iteration')
plt.ylim(-7.75,-7.74)
plt.yticks(np.arange(-7.75,-7.74,.0005),minor=True)
plt.grid(visible=True,which='both',axis='y')
plt.show()

# saves data to .csv fiile

In [72]:
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
with open("iterations_Experiments_to_Bring_Rebal-p32_r1_m7-17"  + formatted_time +  '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    for i in range(len(scores_sci)):
        arr = []
        arr.append(scores_sci[i])
        arr.append(seeds_sci[i])
        for j in outputs_sci[i]:
            arr.append(j)
        wr.writerow(arr) 

# Find the best scoring seed

In [ ]:
print(len(seeds_sci))
sorted_scores = np.argsort(scores_sci)
print(scores_sci[sorted_scores[0]])
print(seeds_sci[sorted_scores[0]])